### In this file, we'll try to convert all lyrics into their phonetical representations. This conversion involves the use of an external website "http://upodn.com/phon.php" which turns any given English text to its IPA (International Phonetic Alphabet) format. Several issues and characteristics regarding this conversion can be summarized with  the following points:
- The website assumes that any given text is pronounced in American English. This might be a problem for songs that are rhymed by British English pronounciation of words. However since we do not know the exact origin of any given song for sure, we had to proceed with any of the two above mentioned pronounciation schemes.
- The conversion is imperfect: interjections or exclamations (such as oooh, aah) are often not recognized, named entities are also not recognized mostly, and of course certain non-English characters happen to exist in certain songs are left unconverted.
- As a result of unrecognized characters, uncoverted words and so on, there are more than 200 different characters in the whole corpora. The list of these characters are given in this document later on.
- 

#### Start by importing necessary webscraping packages and other packages that will be used throughout the document

In [2]:
try:
    import urllib.request as urllib2
except ImportError:
    import urllib2 
from bs4 import BeautifulSoup

In [3]:
import requests

In [4]:
import pickle
def writePickle( Variable, fname):
    filename = fname +".pkl"
    f = open("pickle_vars/"+filename, 'wb')
    pickle.dump(Variable, f, protocol=4)
    f.close()
def readPickle(fname):
    filename = "pickle_vars/"+fname +".pkl" # notice the ../ addition to reach out to variables from the parent directory
    f = open(filename, 'rb')
    obj = pickle.load(f)
    f.close()
    return obj

In [5]:
import time

Read the pre-existing dictionary that maps song ids to lyrics.

In [ ]:
ids_to_lyrics = readPickle("final_IDs_to_Lyrics_dict")

#### The following script handles the convertion process. Since this process takes approximately 12-15 hours, it is better to handle it with another script that runs in the server. In the documents folder, the script named "Rhythmic Features Extraction.py" serves that purpose

In [ ]:
# read the ids to lyrics dict to get song lyrics to be converted to phonemes
ids_to_lyrics = readPickle("final_IDs_to_Lyrics_dict")
length = len(list(ids_to_lyrics.keys()))

print("There are a total of", length, "songs to be converted to phonemes")

# main script that accesses a web page and converts lyrics to phonemes

id2phoneme_dict = dict()
url = 'http://upodn.com/phon.php' # the web site used for phoneme translation. all translations in this script are made in American English
problematic_ids = list()
counter = 0
for ids, lyrics in ids_to_lyrics.items():
    counter += 1
    print(counter, "out of", length, "songs is being processed. The song id being processed now is:", ids)
    lines = lyrics.split('<>')
    text = ""
    # check whether the second line gives song writer info. if so, remove the first two lines
    if lines[1][0:6] == 'Writer' or lines[1][0:7] == 'Writers':
        lines = lines[2:]
    for line in lines[:-1]: # last line is blank in all of the lyrics
        text += line
        text += " ohohohohoh " # we use this a separator of lines. we are sure that this string cannot be converted to any phonetical representation
    # the following two lines handles the form filling in the static website
    post_params = {'intext': text}
    response = requests.post(url, data=post_params)
    soup = BeautifulSoup(response.text, 'html.parser')
    try:
        phoneme_translation = soup.findAll('td', {'align': "left"})[0].get_text() # this website holds the transcript in a td box that is aligned left
        id2phoneme_dict[ids] = phoneme_translation
    except:
        problematic_ids.append(ids)        
    
    if counter%150==0: # for every 150 songs translated, save the dictionary and wait for 15 seconds to give a break to the website
        writePickle(id2phoneme_dict, "final_IDs_to_Phonemes_dict")
        print(phoneme_translation)
        time.sleep(15)
        


print("The last phase of translation: Writing the phoneme dictionaries to pickle files...")
writePickle(id2phoneme_dict, "final_IDs_to_Phonemes_dict")

Read the pickle file that is obtained by running the scraping script on the server and contains the dictionary mapping song ids to phonetical representations

In [ ]:
final_IDs_to_Phonemes_dict = readPickle("final_IDs_to_Phonemes_dict")

# an example
final_IDs_to_Phonemes_dict["81541"]

#### We now apply some processing to the transcriptions we obtained


First gather all the unique characters used in the complete translated corpora and map them to indices


In [ ]:
all_phoneme_set = set()
for item in list(final_IDs_to_Phonemes_dict.values()):
    all_phoneme_set.update(set(item))
    
print("There are", len(all_phoneme_set), "different phonetic symbols and characters in the whole corpus of lyrics")
print(all_phoneme_set)

There are 243 different phonetic symbols and characters in the whole corpus of lyrics
{'î', ',', 'ɛ', 'ʧ', 'À', 'z', '歌', '⁰', '海', 'y', '9', 'g', 'з', '¿', '̀', 'し', 'う', 'る', '0', '手', '愛', 'び', 'ɒ', 'の', 'Ó', 'て', 't', 'л', '❤', 'k', 'ō', 'は', 'Ô', 'â', 'е', 'ŷ', 'á', '—', '抰', '‛', 'w', '浜', 'µ', 'М', 'н', 'c', 'こ', 'ː', 'м', 'u', 'ʤ', 'q', 'ç', '³', '憂', '´', '́', 'Ü', '達', 'ɑ', '“', 'ô', '‘', 'i', '’', 'Б', '™', 'о', 'Â', 'を', '\x98', '方', 'й', '行', 'ら', 'ь', '飗', 'マ', '抌', 'e', 'œ', 'к', '£', '×', '\x80', 'Ч', 'Î', '白', 'ò', '彼', '\n', 'と', 'れ', 'ま', 'h', 'ä', 'b', 'ʒ', 'x', '歩', '抎', '¦', 'и', "'", '8', '悲', 's', 'j', '\u200a', 'у', 'd', '恋', '\u2028', '散', 'ɔ', '©', '秋', 'È', 'い', '¨', '\\', '·', 'o', '‿', '抦', '°', 'í', 'ɪ', 'à', 'v', 'ﬂ', '–', '木', 'û', 'ì', 'Á', 'ц', '、', 'ʊ', '\x85', '…', '²', 'ə', 'さ', 'ē', 'き', '-', 'か', ' ', '5', '探', 'ð', 'す', 'Ò', 'è', 'а', 'に', 'ö', 'ʃ', '¡', 'с', 'ó', 'っ', 'я', 'ã', 'ê', 'a', '”', '€', 'ぎ', 'が', 'õ', '№', 'ú', 'ﬁ', '1', 'ṗ', 'な', 'B', 'в', 'ë', 'た', 'ñ', '|', '抯', 'É', '2', 'п', '過', '抮', 'т', 'é', 'р', 'æ', '3', 'n', '¢', '‑', 'r', '¼', 'Š', '\t', 'ŋ', '葉', '人', 'ɡ', 'Н', '鈚', 'θ', '¹', 'ø', 'よ', 'つ', '®', 'l', '.', '¯', '½', '\xa0', 'б', 'Õ', 'p', 'ï', 'д', '4', '抳', 'ü', 'く', 'f', 'Ž', 'ß', '7', 'Ö', 'm', 'そ', '️', 'S', '6', '\x7f', 'ù', '抣', 'ā', 'š'}

Then use dictionaries to map each phonetical character used to ids

In [ ]:
unique_phoneme_list = list(all_phoneme_set)
unique_phoneme_list.insert(0,"PADDING")

In [ ]:
PHO2id = {pho: index for index, pho in enumerate(unique_phoneme_list)}
id2PHO = {index: pho for index, pho in enumerate(unique_phoneme_list)}
print(PHO2id)

{'î': 1, ',': 2, 'ʧ': 4, 'À': 5, 'z': 6, 'a': 167, '歌': 7, '海': 9, 'w': 41, 'ɛ': 3, '9': 11, 'し': 16, 'g': 12, '̀': 15, 'る': 18, 'М': 44, 'Ž': 230, '0': 19, '愛': 21, 'び': 22, 'ɒ': 23, 'ø': 211, 'の': 24, "'": 104, 't': 27, 'л': 28, '❤': 29, 'k': 30, '過': 189, 'ō': 31, 'は': 32, 'Ô': 33, 'â': 34, 'е': 35, 'ŷ': 36, 'á': 37, '抰': 39, '6': 238, '‛': 40, '浜': 42, 'µ': 43, '⁰': 8, 'н': 45, 'c': 46, '歩': 100, 'こ': 47, 'ː': 48, 'u': 50, 'ʤ': 51, '³': 54, '憂': 55, '́': 57, 'Ü': 58, 'ô': 62, 'ɑ': 60, '“': 61, 'b': 97, '‘': 63, 'i': 64, '’': 65, 'Б': 66, 'о': 68, '\x98': 71, '抳': 226, 'з': 13, '方': 72, 'й': 73, 'ら': 75, 'ь': 76, 'マ': 78, '£': 83, 'œ': 81, '\x80': 85, '×': 84, 'Ч': 86, '白': 88, 'ò': 89, 'y': 10, '\n': 91, 'と': 92, 'れ': 93, '\xa0': 219, 'ま': 94, 'h': 95, 'ä': 96, 'ʒ': 98, 'x': 99, '抎': 101, '¦': 102, 'и': 103, '8': 105, '達': 59, '悲': 106, 's': 107, 'ā': 242, '\u200a': 109, 'у': 110, 'd': 111, '恋': 112, '\u2028': 113, '散': 114, 'ɔ': 115, '©': 116, '”': 168, '秋': 117, 'ì': 135, 'È': 118, 'п': 188, '´': 56, 'p': 222, 'い': 119, '¨': 120, '\\': 121, '·': 122, 'o': 123, '°': 126, '‿': 124, 'É': 186, '抦': 125, 'か': 148, 'í': 127, 'ɪ': 128, '手': 20, 'à': 129, 'j': 108, 'v': 130, '–': 132, 'û': 134, '™': 67, 'q': 52, 'ü': 227, 'ц': 137, '、': 138, '\x7f': 239, 'ʊ': 139, 'ē': 145, 'PADDING': 0, '…': 141, '²': 142, 'ə': 143, 'さ': 144, 'く': 228, 'Â': 69, 'き': 146, '抌': 79, '-': 147, ' ': 149, '5': 150, 'f': 229, 'ð': 152, 'す': 153, 'Ò': 154, 'è': 155, 'Ó': 25, 'а': 156, 'e': 80, 'に': 157, 'ö': 158, 'ʃ': 159, 'を': 70, '½': 218, 'с': 161, 'ó': 162, 'っ': 163, 'я': 164, 'ã': 165, 'ê': 166, '€': 169, 'ぎ': 170, 'が': 171, 'よ': 212, 'õ': 172, '№': 173, '木': 133, 'é': 192, 'ﬁ': 175, '1': 176, 'ṗ': 177, 'な': 178, 'B': 179, 'в': 180, 'ë': 181, 'ñ': 183, '|': 184, '抯': 185, '行': 74, '2': 187, '抮': 190, 'n': 196, 'т': 191, 'ﬂ': 131, 'р': 193, 'æ': 194, '3': 195, '飗': 77, '¢': 197, '¯': 217, 'r': 199, '‑': 198, 'Š': 201, '\t': 202, 'ŋ': 203, '¿': 14, 'к': 82, '葉': 204, '人': 205, 'ɡ': 206, 'Á': 136, 'Н': 207, '鈚': 208, 'θ': 209, '¹': 210, '¡': 160, 'つ': 213, '®': 214, 'l': 215, '¼': 200, '.': 216, '\x85': 140, 'б': 220, 'Õ': 221, 'Î': 87, 'д': 224, '4': 225, 'š': 243, 'う': 17, '—': 38, 'ï': 223, 'ç': 53, 'ß': 231, '7': 232, 'Ö': 233, 'm': 234, 'そ': 235, '️': 236, 'S': 237, 'ú': 174, '彼': 90, 'て': 26, 'ù': 240, '探': 151, '抣': 241, 'м': 49, 'た': 182}

In [ ]:
writePickle(PHO2id, "indexing/PHO2id")
writePickle(id2PHO, "indexing/id2PHO")

#### Continue with processing the phonetical transcripts to strip them from unnecessary whitespaces etc.

In [ ]:
final_IDs_to_ProcessedPhonemes_dict = dict()
for ids, item in final_IDs_to_Phonemes_dict.items():
    lines = item.split("ohohohohoh")
    combined_lines = list()
    for line in lines:
        if line.strip() != "":
            combined_lines.append(line.strip())
    final_IDs_to_ProcessedPhonemes_dict[ids] = combined_lines

#### Now we will convert all songs to phoneme ids and allocate them to training, development and test sets for the CNN model

#### The first application will be based on purely phonetic characters, without any emphasis on specific rhyme schemes


First, write each song as a list of lists, where each sublist is a combination of splitted characters <br>
Example: "i love // and i hate" will be converted to [['i', ' ', 'l', 'o', 'v', 'e'], ['a', 'n', 'd', '','i', ' ', 'h', 'a', 't', 'e']]

In [ ]:
def split(string): 
    return [char for char in string]  

ids2splitted_phonemes_dict = dict()
for song_id, phonetics in final_IDs_to_ProcessedPhonemes_dict.items():
    ids2splitted_phonemes_dict[song_id] = list()
    for line in phonetics:
        ids2splitted_phonemes_dict[song_id].append(split(line))

In [ ]:
# an example
print(ids2splitted_phonemes_dict["422792"])

Then find the maximum song length and the maximum line length in the whole corpus

In [ ]:
max_song = 0
max_line = 0
for song_id, phonemes in ids2splitted_phonemes_dict.items():
    song_length = len(phonemes)
    if song_length > max_song:
        max_song = len(phonemes)
    for line in phonemes:
        line_length = len(line)
        if line_length > max_line:
            max_line = len(line)
    

The maximum length of any given song doesn't change and is fixed at 100 as defined earlier. However as for the maximum length of lines, since we're now working with characters instead of tokens, we have as much as 183 characters in a line.

Now it is time to take the songs allocated for each datasets (train, test, dev), and convert matching lyrics to padded version of phonetical symbols

In [ ]:
import numpy as np
import pandas as pd
from collections import OrderedDict


PHO2id = readPickle("indexing/PHO2id")
Artist2id = readPickle("indexing/Artist2id")
ID_to_PHO = ids2splitted_phonemes_dict

# import also the splitted datasets
train_df = readPickle("train_df")
dev_df = readPickle("dev_df")
test_df = readPickle("test_df")

# the following function carries out the transformation. notice the number 1 in the naming, since another version of\
# this function will be used later on for other types of phonetical representations
def PHO1_generator(dataframe): # pick any of the dataframes; -test, -train or -dev
    PHO_dict = dataframe.to_dict('list')
    sorted_PHO_dict = OrderedDict(sorted(PHO_dict.items(), key=lambda v: v, reverse=True))
    artists = list()
    songs = list()
    for artist, song_ID_list in sorted_PHO_dict.items():
        for song_ID in song_ID_list:
            PHO_song = list()
            artists.append(Artist2id[artist])
            song = ID_to_PHO[song_ID]
            while len(song) != max_song:
                song.append(["PADDING"])
            for line in song:
                while len(line) != max_line:
                    line.append("PADDING")
            for line in song:
                #print("line is",line)
                PHO_line = list()
                for tag in line:
                    #print(tag)
                    PHO_line.append(PHO2id[tag])
                PHO_song.append(PHO_line)
            songs.append(PHO_song)
    return songs, artists

# using the function, form the datasets in python list format
# again for naming, use the index '1', since we will have other phonetic transcription versions later on
train_PHO1_input_data, train_PHO1_labels = PHO1_generator(train_df)
print("Training data finished, continuing with development data...")
dev_PHO1_input_data, dev_PHO1_labels = PHO1_generator(dev_df)
print("Development data finished, continuing with test data...")
test_PHO1_input_data, test_PHO1_labels = PHO1_generator(test_df)
print("Test data finished, continuing with pickle file recording...")




# in the end store these as pickle variables for later use
writePickle(train_PHO1_input_data, "cnn_data_inputs/train_PHO1_input_data")
writePickle(train_PHO1_labels, "cnn_data_inputs/train_PHO1_labels")
writePickle(dev_PHO1_input_data, "cnn_data_inputs/dev_PHO1_input_data")
writePickle(dev_PHO1_labels, "cnn_data_inputs/dev_PHO1_labels")
writePickle(test_PHO1_input_data, "cnn_data_inputs/test_PHO1_input_data")
writePickle(test_PHO1_labels, "cnn_data_inputs/test_PHO1_labels")

print("An example of training phonetic input data is:", train_PHO1_input_data[0])
print("The first training phonetic label is", train_PHO1_labels[0])

In [ ]:
# start with reading these dataset variables from pickle files
train_PHO1_input_data = readPickle("cnn_data_inputs/train_PHO1_input_data")
train_PHO1_labels = readPickle("cnn_data_inputs/train_PHO1_labels")
dev_PHO1_input_data = readPickle("cnn_data_inputs/dev_PHO1_input_data")
dev_PHO1_labels = readPickle("cnn_data_inputs/dev_PHO1_labels")
test_PHO1_input_data = readPickle("cnn_data_inputs/test_PHO1_input_data")
test_PHO1_labels = readPickle("cnn_data_inputs/test_PHO1_labels")

In [ ]:
# convert all of them to numpy arrays, so that they can be used in keras
import numpy as np

train_PHO1_input_data = np.array(train_PHO1_input_data)
train_PHO1_labels = np.array(train_PHO1_labels)
dev_PHO1_input_data = np.array(dev_PHO1_input_data)
dev_PHO1_labels = np.array(dev_PHO1_labels)
test_PHO1_input_data = np.array(test_PHO1_input_data)
test_PHO1_labels = np.array(test_PHO1_labels)

In [ ]:
# for all the input data, we have to normalize the data points to an interval between 0 and 1, 
# and convert everything to floating numbers

print(np.amax(train_PHO1_input_data))
print(np.amax(test_PHO1_input_data))
print(np.amax(dev_PHO1_input_data))

train_PHO1_input_data = train_PHO1_input_data.astype('float32') / np.amax(train_PHO1_input_data)
dev_PHO1_input_data = dev_PHO1_input_data.astype('float32') / np.amax(dev_PHO1_input_data)
test_PHO1_input_data = test_PHO1_input_data.astype('float32') / np.amax(test_PHO1_input_data)

In [ ]:
# check whether things work fine

print(train_PHO1_input_data[0])
print(train_PHO1_labels[0])
print(train_PHO1_input_data.shape)

In [ ]:
# reshape the inputs into desired format

X_train_PHO1 = train_PHO1_input_data.reshape(len(train_PHO1_input_data),max_song,max_line,1)
X_dev_PHO1 = dev_PHO1_input_data.reshape(len(dev_PHO1_input_data),max_song,max_line,1)
X_test_PHO1 = test_PHO1_input_data.reshape(len(test_PHO1_input_data),max_song,max_line,1)

# check shape
print(X_train_PHO1.shape)


In [ ]:
# then we need to turn the labels into categorical values

from keras.utils import to_categorical

y_train_PHO1 = to_categorical(train_PHO1_labels)
y_dev_PHO1 = to_categorical(dev_PHO1_labels)
y_test_PHO1 = to_categorical(test_PHO1_labels)

print(y_train_PHO1)

In [ ]:
# Record all of the formatted final input and outputs into pickle files

writePickle(X_train_PHO1,"cnn_data_inputs/PHO1_Keras/X_train_PHO1")
writePickle(X_dev_PHO1,"cnn_data_inputs/PHO1_Keras/X_dev_PHO1")
writePickle(X_test_PHO1,"cnn_data_inputs/PHO1_Keras/X_test_PHO1")
writePickle(y_train_PHO1,"cnn_data_inputs/PHO1_Keras/y_train_PHO1")
writePickle(y_dev_PHO1,"cnn_data_inputs/PHO1_Keras/y_dev_PHO1")
writePickle(y_test_PHO1,"cnn_data_inputs/PHO1_Keras/y_test_PHO1")

#### Now apply the same process for a similar reduced version

In [ ]:
import heapq, random
ID_to_Partial_Rhymes = readPickle("ID_to_Partial_Rhymes")

counter = 0
songID_to_reduced_10_10_phonemes = dict()
for song_id, song in ID_to_Partial_Rhymes.items():
    counter +=1
    print(counter, song_id)
    song_line_ruler = list()
    for line in song:
        line_index = song.index(line)
        importants = sum(map(lambda i: i > 0.5, line))
        song_line_ruler.append((line_index, importants))
        song_line_ruler = list(set(song_line_ruler)) # remove duplicate lines
        song_line_ruler.sort(key=lambda x: x[1], reverse=True) # sort by the number of most important rhymes
    reduced_song = list()
    song_line_ruler = song_line_ruler[0:10] # take the best 10, sorted by rhyme importance
    song_line_ruler.sort(key=lambda x: x[0]) # sort them back in shape wrt their original places in the song
    # if the line has more than 10 phonemes, get the most important ones
    for item in song_line_ruler:
        reduced_song.append(song[item[0]])
            
    # when the song has less than 10 lines, complete the song with 0.0 lines towards the end
    if len(reduced_song) < 10:
        diff = 10 - len(reduced_song)
        for _ in range(diff):
            reduced_song.append([0.0])
    # then shrink (remove the smallest values first) or expand lines (with random 0's) that doesn't have exactly 10 phoneme values
    for new_line in reduced_song:
        if len(new_line) > 10:
            diff = len(new_line) - 10
            to_go = heapq.nsmallest(diff, new_line)
            for item in to_go[:]:
                new_line.remove(item)
        elif len(new_line) < 10:
            length = len(new_line)
            diff = 10 - length
            for _ in range(diff):
                new_line.insert(random.randrange(0,length,1),0.0)
    songID_to_reduced_10_10_phonemes[song_id] = reduced_song


In [ ]:
import numpy as np
import pandas as pd
from collections import OrderedDict


Artist2id = readPickle("indexing/Artist2id")

# import also the splitted datasets
train_df = readPickle("train_df")
dev_df = readPickle("dev_df")
test_df = readPickle("test_df")

# the following function carries out the transformation. 
def PHO2_generator(dataframe, rhyme_collection): # pick any of the dataframes; -test, -train or -dev
    PHO_dict = dataframe.to_dict('list')
    sorted_PHO_dict = OrderedDict(sorted(PHO_dict.items(), key=lambda v: v, reverse=True))
    artists = list()
    songs = list()
    for artist, song_ID_list in sorted_PHO_dict.items():
        for song_ID in song_ID_list:
            PHO_song = list()
            artists.append(Artist2id[artist])
            songs.append(rhyme_collection[song_ID])
    return songs, artists

# using the function, form the datasets in python list format
# again for naming, use the index '1', since we will have other phonetic transcription versions later on
train_PHO_10_10_input_data, train_PHO_10_10_labels = PHO2_generator(train_df, songID_to_reduced_10_10_phonemes)
print("Training data finished, continuing with development data...")

dev_PHO_10_10_input_data, dev_PHO_10_10_labels = PHO2_generator(dev_df, songID_to_reduced_10_10_phonemes)
print("Development data finished, continuing with test data...")

test_PHO_10_10_input_data, test_PHO_10_10_labels = PHO2_generator(test_df, songID_to_reduced_10_10_phonemes)
print("Test data finished, continuing with pickle file recording...")



# in the end store these as pickle variables for later use
writePickle(train_PHO_10_10_input_data, "cnn_data_inputs/train_PHO_10_10_input_data")
writePickle(train_PHO_10_10_labels, "cnn_data_inputs/train_PHO_10_10_labels")
writePickle(dev_PHO_10_10_input_data, "cnn_data_inputs/dev_PHO_10_10_input_data")
writePickle(dev_PHO_10_10_labels, "cnn_data_inputs/dev_PHO_10_10_labels")
writePickle(test_PHO_10_10_input_data, "cnn_data_inputs/test_PHO_10_10_input_data")
writePickle(test_PHO_10_10_labels, "cnn_data_inputs/test_PHO_10_10_labels")


print("An example of training phonetic input data is:", train_PHO_10_10_input_data[0])
print("The first training phonetic label is", train_PHO_10_10_labels[0])


In [ ]:
train_PHO_10_10_input_data = np.array(train_PHO_10_10_input_data)
train_PHO_10_10_labels = np.array(train_PHO_10_10_labels)
dev_PHO_10_10_input_data = np.array(dev_PHO_10_10_input_data)
dev_PHO_10_10_labels = np.array(dev_PHO_10_10_labels)
test_PHO_10_10_input_data = np.array(test_PHO_10_10_input_data)
test_PHO_10_10_labels = np.array(test_PHO_10_10_labels)

X_train_PHO_10_10 = train_PHO_10_10_input_data.reshape(len(train_PHO_10_10_input_data),train_PHO_10_10_input_data.shape[1],train_PHO_10_10_input_data.shape[2],1)
X_dev_PHO_10_10 = dev_PHO_10_10_input_data.reshape(len(dev_PHO_10_10_input_data),dev_PHO_10_10_input_data.shape[1],dev_PHO_10_10_input_data.shape[2],1)
X_test_PHO_10_10 = test_PHO_10_10_input_data.reshape(len(test_PHO_10_10_input_data),test_PHO_10_10_input_data.shape[1],test_PHO_10_10_input_data.shape[2],1)

# then we need to turn the labels into categorical values
from keras.utils import to_categorical

y_train_PHO_10_10 = to_categorical(train_PHO_10_10_labels)
y_dev_PHO_10_10 = to_categorical(dev_PHO_10_10_labels)
y_test_PHO_10_10 = to_categorical(test_PHO_10_10_labels)

writePickle(X_train_PHO_10_10,"cnn_data_inputs/PHO1_Keras/X_train_PHO_10_10")
writePickle(X_dev_PHO_10_10,"cnn_data_inputs/PHO1_Keras/X_dev_PHO_10_10")
writePickle(X_test_PHO_10_10,"cnn_data_inputs/PHO1_Keras/X_test_PHO_10_10")
writePickle(y_train_PHO_10_10,"cnn_data_inputs/PHO1_Keras/y_train_PHO_10_10")
writePickle(y_dev_PHO_10_10,"cnn_data_inputs/PHO1_Keras/y_dev_PHO_10_10")
writePickle(y_test_PHO_10_10,"cnn_data_inputs/PHO1_Keras/y_test_PHO_10_10")

print(X_train_PHO_10_10.shape)

### From the this moment on, we will use the Model scripts to construct our models, using the pickle input variables right above